# MIPT Train Contest EDA

- age — AAGE
- class of worker — ACLSWKR
- industry code — ADTIND
- occupation code — ADTOCC
- education — AHGA
- wage per hour — AHRSPAY
- enrolled in edu inst last wk — AHSCOL
- marital status — AMARITL
- major industry code — AMJIND
- major occupation code — AMJOCC
- mace — ARACE
- hispanic origin — AREORGN
- sex — ASEX
- member of a labor union — AUNMEM
- reason for unemployment — AUNTYPE
- full or part time employment stat — AWKSTAT
- capital gains — CAPGAIN
- capital losses — CAPLOSS
- divdends from stocks — DIVVAL
- tax filer status — FILESTAT
- region of previous residence — GRINREG
- state of previous residence — GRINST
- detailed household and family stat — HHDFMX
- detailed household summary in household — HHDREL
- instance weight — MARSUPWT
- migration code-change in msa — MIGMTR1
- migration code-change in reg — MIGMTR3
- migration code-move within reg — MIGMTR4
- live in this house 1 year ago — MIGSAME
- migration prev res in sunbelt — MIGSUN
- num persons worked for employer — NOEMP
- family members under — 18 PARENT
- country of birthfather — PEFNTVTY
- country of birthmother — PEMNTVTY
- country of birthself — PENATVTY
- citizenship — PRCITSHP
- own business or self employed — SEOTR
- taxable income amount — TAXINC
- fill inc questionnaire for veteran's admin — VETQVA
- veterans benefits — VETYN
- weeks worked in year — WKSWORK

In [1]:
import numpy as np
import pandas as pd
import pandas_profiling

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import normalize

from sklearn.metrics import accuracy_score 

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


RANDOM_STATE = 42

numeric = ['AAGE', 'ADTIND', 'ADTOCC', 'AHRSPAY', 'CAPGAIN', 'CAPLOSS',
          'DIVVAL', 'MARSUPWT', 'NOEMP', 'SEOTR', 'VETQVA', 'VETYN']
categ = ['ACLSWKR', 'AHGA', 'AHSCOL', 'AMARITL', 'AMJIND', 'AMJOCC',
       'ARACE', 'AREORGN', 'ASEX', 'AUNMEM', 'AUNTYPE', 'FILESTAT',
       'GRINREG', 'GRINST', 'HHDFMX', 'HHDREL', 'MIGMTR1', 'MIGMTR3',
       'MIGMTR4', 'MIGSAME', 'MIGSUN', 'PARENT', 'PEFNTVTY', 'PEMNTVTY',
       'PENATVTY', 'PRCITSHP', 'TAXINC']
drop = ['AWKSTAT', 'WKSWORK']

In [2]:
data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

# data.icol()

data.loc[data['CLASS']=='50000+', 'CLASS'] = 1
data.loc[data['CLASS']=='-50000', 'CLASS'] = 0
data['CLASS'] = data['CLASS'].astype(int)
data['CLASS'] = data['CLASS'].astype(bool)

data.drop(categ, axis=1)
test_data.drop(categ, axis=1)

test_index = test_data['Id']
test_data.drop('Id', axis=1)

train_data = data.drop('CLASS', axis=1)
target = data['CLASS']

train_data['status'] = 'train'
test_data['status'] = 'test'

all_data = pd.concat([train_data, test_data])
all_data.head(2)

,AAGE,ACLSWKR,ADTIND,ADTOCC,AHGA,AHRSPAY,AHSCOL,AMARITL,AMJIND,AMJOCC,...,PEFNTVTY,PEMNTVTY,PENATVTY,PRCITSHP,SEOTR,TAXINC,VETQVA,VETYN,WKSWORK,status
0,43,LocalGovernment,43,10,Mastersdegree(MAMSMEngMEdMSWMBA),0,Not_in_universe,Married-civilianspousepresent,Education,Professionalspecialty,...,United-States,United-States,United-States,Native-BornintheUnitedStates,0,Not_in_universe,2,52,94,train
1,0,Not_in_universe,0,0,Children,0,Not_in_universe,Nevermarried,Not_in_universeorchildren,Not_in_universe,...,United-States,United-States,United-States,Native-BornintheUnitedStates,0,Not_in_universe,0,0,94,train


In [3]:
# pandas_profiling.ProfileReport(data)

**AWKSTAT** - constant, can be dropped

**WKSWORK** - constant, can be dropped

In [4]:
# pandas_profiling.ProfileReport(test_data)

In [3]:
# Prepare categ
for col in categ:
    if col=='status':
        continue
    enc = LabelEncoder()
    all_data[col] = enc.fit_transform(all_data[col])
one_hot_enc = OneHotEncoder(categorical_features='all', sparse=False)
categ_encoded = one_hot_enc.fit_transform(all_data[categ])

all_data[numeric] = normalize(all_data[numeric], axis=0)

X_train_data = np.hstack([all_data[numeric][all_data['status']=='train'], categ_encoded[:10000,:]])
X_test_data = np.hstack([all_data[numeric][all_data['status']=='test'], categ_encoded[10000:, :]])

y = target

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X_train_data, y,
                                                    test_size=0.2, random_state=RANDOM_STATE)

In [5]:
log_reg_clf = LogisticRegression()

# Set the parameters by cross-validation

# best l1 , 0.8695
params = {
    'penalty': ['l1'], #['l2']]
    'C': [100],
    'solver': ['liblinear'], # ['newton-cg', 'lbfgs', 'sag']
#     'tol': [0.001],
#     'intercept_scaling': np.linspace(0.1, 100, 10)
}



clf_cv = GridSearchCV(
    LogisticRegression(n_jobs=-1, random_state=RANDOM_STATE,
                  max_iter=100), params)

# log_reg_clf.fit(X_train, y_train)

clf_cv.fit(X_train, y_train)

print 'score', accuracy_score(y_test, clf_cv.predict(X_test))

clf_cv.best_estimator_

score 0.8695


LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
real_pred = clf_cv.predict_proba(X_test_data)[:, 1]

In [17]:
clf = LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

clf.fit(X_train_data, target)
real_pred = clf.predict_proba(X_test_data)[:, 1]

ans = pd.DataFrame({'Id': test_index, 'Prediction': real_pred})
ans.to_csv('ans.csv', index=False)

In [11]:
test_data.head()

,AAGE,ACLSWKR,ADTIND,ADTOCC,AHGA,AHRSPAY,AHSCOL,AMARITL,AMJIND,AMJOCC,...,PENATVTY,PRCITSHP,SEOTR,TAXINC,VETQVA,VETYN,WKSWORK,Id,status,CLASS
0,28,Private,33,16,Associatesdegree-occup/vocational,0,Not_in_universe,Nevermarried,Retailtrade,Sales,...,United-States,Native-BornintheUnitedStates,0,Not_in_universe,2,52,94,1,test,0
1,41,Private,11,14,Associatesdegree-academicprogram,0,Not_in_universe,Married-civilianspousepresent,Manufacturing-durablegoods,Techniciansandrelatedsupport,...,United-States,Native-BornintheUnitedStates,0,Not_in_universe,2,52,94,2,test,0
2,37,Not_in_universe,0,0,Somecollegebutnodegree,0,Not_in_universe,Married-civilianspousepresent,Not_in_universeorchildren,Not_in_universe,...,United-States,Native-BornintheUnitedStates,0,Not_in_universe,2,0,94,3,test,0
3,43,Private,25,18,Highschoolgraduate,0,Not_in_universe,Married-civilianspousepresent,Manufacturing-nondurablegoods,Sales,...,United-States,Native-BornintheUnitedStates,0,Not_in_universe,2,52,94,4,test,0
4,36,Private,29,17,Bachelorsdegree(BAABBS),0,Not_in_universe,Married-civilianspousepresent,Transportation,Sales,...,United-States,Native-BornintheUnitedStates,0,Not_in_universe,2,52,94,5,test,0


In [ ]:
# score: 0.18496

In [ ]:
#############################

In [ ]:
coefs = []

for i, coef in enumerate(clf_cv.best_estimator_.coef_[0]):
    coefs.append((np.abs(coef), i))
    
coefs = sorted(coefs, reverse=True)

for coef in coefs:
    print coef
    
best_features = coefs[:25]
best_indecies = map(lambda it: it[1], best_features)

X_train_data[:, best_indecies]

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_train_data[:, best_indecies], y,
                                                    test_size=0.2, random_state=RANDOM_STATE)

In [34]:
log_reg_clf = LogisticRegression()

# Set the parameters by cross-validation

# best l1 , 0.8695
params = {
    'penalty': ['l2'], #['l2']]
    'C': [0.001, 0.1, 10, 50, 100, 100],
    'solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag']
#     'tol': [0.001],
#     'intercept_scaling': np.linspace(0.1, 100, 10)
}


clf_cv = GridSearchCV(
    LogisticRegression(n_jobs=-1, random_state=RANDOM_STATE,
                  max_iter=100), params)

# log_reg_clf.fit(X_train, y_train)

clf_cv.fit(X_train, y_train)

print 'score', accuracy_score(y_test, clf_cv.predict(X_test))


D:\soft\dev\anaconda\lib\site-packages\sklearn\linear_model\sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


score 0.8215


In [55]:
from sklearn.ensemble import RandomForestClassifier


# Set the parameters by cross-validation

# best l1 , 0.8695
params = {
    'n_estimators': [1000],
    'criterion': ['gini'], # 'entropy']
    'max_features': ['sqrt', 'log2'],
#     'max_depth': [5, 8, 10],
#     'min_samples_split': [2, 3, 5],
#     'min_samples_leaf': [1, 3, 5]
    
}



clf_cv = GridSearchCV(
    RandomForestClassifier(n_jobs=-1, random_state=RANDOM_STATE), params)

# log_reg_clf.fit(X_train, y_train)

clf_cv.fit(X_train, y_train)

print 'score', accuracy_score(y_test, clf_cv.predict(X_test))

clf_cv.best_estimator_

score 0.8665


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [56]:
real_rf_pred = clf_cv.predict_proba(X_test_data)[:, 1]

ans = pd.DataFrame({'Id': test_index, 'Prediction': real_rf_pred})
ans.to_csv('ans.csv', index=False)

In [57]:
ans = pd.DataFrame({'Id': test_index, 'Prediction': (real_pred+real_rf_pred)/2.})